<a href="https://colab.research.google.com/github/Rahulkumar6591/NewCoding/blob/main/INT354_Project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import pandas as pd

In [5]:
movies = pd.read_csv('/content/movies.csv',sep=';',encoding='latin-1').drop('Unnamed: 3',axis=1)
print('Shape of this dataset :',movies.shape)
movies.head()

Shape of this dataset : (3883, 3)


,movieId,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
ratings = pd.read_csv('/content/ratings.csv',sep=';')
print('Shape of this dataset :',ratings.shape)
ratings.head()

Shape of this dataset : (145195, 4)


,userId,movieId,rating,timestamp
0,1,1193.0,5.0,978300760.0
1,1,661.0,3.0,978302109.0
2,1,914.0,3.0,978301968.0
3,1,3408.0,4.0,978300275.0
4,1,2355.0,5.0,978824291.0


In [4]:
users = pd.read_csv('/content/users.csv',sep=';')
print('Shape of this dataset :',users.shape)
users.head()

Shape of this dataset : (6040, 5)


,userId,gender,age,occupation,zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,2460
4,5,M,25,20,55455


In [8]:
rating_pivot = ratings.pivot_table(values='rating',columns='userId',index='movieId').fillna(0)
print('Shape of this pivot table :',rating_pivot.shape)
rating_pivot.head()

Shape of this pivot table : (3555, 1698)


userId,1,2,3,4,5,6,7,8,9,10,...,1689,1690,1691,1692,1693,1694,1695,1696,1697,1698
movieId,,,,,,,,,,,,,,,,,,,,,
1.0,5.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,5.0,5.0,...,0.0,0.0,0.0,0.0,0.0,3.0,5.0,0.0,5.0,0.0
2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,4.0,0.0,0.0,0.0,2.0,3.0,0.0,0.0,0.0,0.0
3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
from sklearn.neighbors import NearestNeighbors
nn_algo = NearestNeighbors(metric='cosine')
nn_algo.fit(rating_pivot)

NearestNeighbors(metric='cosine')

In [10]:
class Recommender:
    def __init__(self):
        # This list will stored movies that called atleast ones using recommend_on_movie method
        self.hist = []
        self.ishist = False # Check if history is empty

    # This method will recommend movies based on a movie that passed as the parameter
    def recommend_on_movie(self,movie,n_reccomend = 5):
        self.ishist = True
        movieid = int(movies[movies['title']==movie]['movieId'])
        self.hist.append(movieid)
        distance,neighbors = nn_algo.kneighbors([rating_pivot.loc[movieid]],n_neighbors=n_reccomend+1)
        movieids = [rating_pivot.iloc[i].name for i in neighbors[0]]
        recommeds = [str(movies[movies['movieId']==mid]['title']).split('\n')[0].split('  ')[-1] for mid in movieids if mid not in [movieid]]
        return recommeds[:n_reccomend]

    # This method will recommend movies based on history stored in self.hist list
    def recommend_on_history(self,n_reccomend = 5):
        if self.ishist == False:
            return print('No history found')
        history = np.array([list(rating_pivot.loc[mid]) for mid in self.hist])
        distance,neighbors = nn_algo.kneighbors([np.average(history,axis=0)],n_neighbors=n_reccomend + len(self.hist))
        movieids = [rating_pivot.iloc[i].name for i in neighbors[0]]
        recommeds = [str(movies[movies['movieId']==mid]['title']).split('\n')[0].split('  ')[-1] for mid in movieids if mid not in self.hist]
        return recommeds[:n_reccomend]

In [11]:
# linitializing the Recommender Object
recommender = Recommender()

In [12]:
# Recommendation based on past watched movies, but the object just initialized. So, therefore no history found
recommender.recommend_on_history()

No history found


In [13]:
# Recommendation based on this movie
recommender.recommend_on_movie('Father of the Bride Part II (1995)')

['Home Alone (1990)',
 'Home Alone 2',
 'City Slickers II',
 'Multiplicity (1996)',
 'Mrs. Doubtfire (1993)']

In [14]:
# Recommendation based on past watched movies, and this time a movie is there in the history.
recommender.recommend_on_history()

['Home Alone (1990)',
 'Home Alone 2',
 'City Slickers II',
 'Multiplicity (1996)',
 'Mrs. Doubtfire (1993)']

In [15]:
# Recommendation based on this movie
recommender.recommend_on_movie('Tigerland (2000)')

['Requiem for a Dream (2000)',
 'Yards, The (1999)',
 'Girlfight (2000)',
 'Psycho Beach Party (2000)',
 'Kissed (1996)']

In [16]:
# Recommendation based on past watched movies, and this time two movies is there in the history.
recommender.recommend_on_history()

['Home Alone 2',
 'Multiplicity (1996)',
 'Home Alone (1990)',
 'City Slickers II',
 'Mrs. Doubtfire (1993)']

In [17]:
# Recommendation based on this movie
recommender.recommend_on_movie('Dracula')

['Dead Man on Campus (1998)',
 'Down Periscope (1996)',
 'Robin Hood',
 'Lord of Illusions (1995)',
 'Tales From the Crypt Presents']

In [18]:
# Recommendation based on past watched movies, and this time three movies is there in the history.
recommender.recommend_on_history()

['Multiplicity (1996)',
 'City Slickers II',
 'Naked Gun 2 1/2',
 'Robin Hood',
 'Home Alone (1990)']

In [19]:
# Recommendation based on this movie
recommender.recommend_on_movie('Money Train (1995)')

['Specialist, The (1994)',
 'Cliffhanger (1993)',
 'Beverly Hills Cop III (1994)',
 'Bad Boys (1995)',
 'Lethal Weapon 3 (1992)']

In [20]:
# Recommendation based on past watched movies, and this time four movies is there in the history.
recommender.recommend_on_history()

['Robin Hood',
 'Batman Forever (1995)',
 'Multiplicity (1996)',
 'Naked Gun 2 1/2',
 'City Slickers II']

In [21]:
# Recommendation based on this movie
recommender.recommend_on_movie('GoldenEye (1995)')

['Tomorrow Never Dies (1997)',
 'World Is Not Enough, The (1999)',
 'Rock, The (1996)',
 'True Lies (1994)',
 'Batman Returns (1992)']

In [22]:
# Recommendation based on past watched movies, and this time five movies is there in the history.
recommender.recommend_on_history()

['Tomorrow Never Dies (1997)',
 'Batman Returns (1992)',
 'World Is Not Enough, The (1999)',
 'Batman Forever (1995)',
 'True Lies (1994)']

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

# Assuming 'movies' is your DataFrame
vectorizer = CountVectorizer(stop_words='english')
genres = vectorizer.fit_transform(movies['genres'].astype(str)).toarray()
contents = pd.DataFrame(genres, columns=vectorizer.get_feature_names_out())
print('Shape of the content table:', contents.shape)
print(contents.head())

Shape of the content table: (3883, 347)
   1919  1956  1963  1968  1974  1977  1978  1979  1980  1981  ...  wight  \
0     0     0     0     0     0     0     0     0     0     0  ...      0   
1     0     0     0     0     0     0     0     0     0     0  ...      0   
2     0     0     0     0     0     0     0     0     0     0  ...      0   
3     0     0     0     0     0     0     0     0     0     0  ...      0   
4     0     0     0     0     0     0     0     0     0     0  ...      0   

   willowbrook  witch  worrying  wrath  years  yellow  yes  york  yu  
0            0      0         0      0      0       0    0     0   0  
1            0      0         0      0      0       0    0     0   0  
2            0      0         0      0      0       0    0     0   0  
3            0      0         0      0      0       0    0     0   0  
4            0      0         0      0      0       0    0     0   0  

[5 rows x 347 columns]


In [24]:
from sklearn.neighbors import NearestNeighbors
nn_algo = NearestNeighbors(metric='cosine')
nn_algo.fit(contents)

NearestNeighbors(metric='cosine')

In [25]:
class Recommender:
    def __init__(self):
        # This list will stored movies that called atleast ones using recommend_on_movie method
        self.hist = []
        self.ishist = False # Check if history is empty

    # This method will recommend movies based on a movie that passed as the parameter
    def recommend_on_movie(self,movie,n_reccomend = 5):
        self.ishist = True
        iloc = movies[movies['title']==movie].index[0]
        self.hist.append(iloc)
        distance,neighbors = nn_algo.kneighbors([contents.iloc[iloc]],n_neighbors=n_reccomend+1)
        recommeds = [movies.iloc[i]['title'] for i in neighbors[0] if i not in [iloc]]
        return recommeds[:n_reccomend]

    # This method will recommend movies based on history stored in self.hist list
    def recommend_on_history(self,n_reccomend = 5):
        if self.ishist == False:
            return print('No history found')
        history = np.array([list(contents.iloc[iloc]) for iloc in self.hist])
        distance,neighbors = nn_algo.kneighbors([np.average(history,axis=0)],n_neighbors=n_reccomend + len(self.hist))
        recommeds = [movies.iloc[i]['title'] for i in neighbors[0] if i not in self.hist]
        return recommeds[:n_reccomend]

In [26]:
# linitializing the Recommender Object
recommender = Recommender()

In [27]:
# Recommendation based on past watched movies, but the object just initialized. So, therefore no history found
recommender.recommend_on_history()

No history found


In [28]:
# Recommendation based on this movie
recommender.recommend_on_movie('Father of the Bride Part II (1995)')

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


['Waiting for Guffman (1996)',
 'Jimmy Hollywood (1994)',
 'Kolya (1996)',
 'Life with Mikey (1993)',
 '8 1/2 Women (1999)']

In [29]:
# Recommendation based on past watched movies, and this time a movie is there in the history.
recommender.recommend_on_history()

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


['Waiting for Guffman (1996)',
 'Jimmy Hollywood (1994)',
 'Kolya (1996)',
 'Life with Mikey (1993)',
 '8 1/2 Women (1999)']

In [30]:
# Recommendation based on this movie
recommender.recommend_on_movie('Tigerland (2000)')

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


['Breaking the Waves (1996)',
 'Jails, Hospitals & Hip-Hop (2000)',
 'They Bite (1996)',
 'Black Tights (Les Collants Noirs) (1960)',
 'Identification of a Woman (Identificazione di una donna) (1982)']

In [31]:
# Recommendation based on past watched movies, and this time two movies is there in the history.
recommender.recommend_on_history()

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


['Sleepover (1995)',
 'Seven Beauties (Pasqualino Settebellezze) (1976)',
 'Virgin Suicides, The (1999)',
 'Man on the Moon (1999)',
 'Two Girls and a Guy (1997)']

In [32]:
# Recommendation based on this movie
recommender.recommend_on_movie('Dracula')

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


['Nemesis 2',
 'Best of the Best 3',
 'Mighty Morphin Power Rangers',
 'Gumby',
 'Die Hard']

In [33]:
# Recommendation based on past watched movies, and this time three movies is there in the history.
recommender.recommend_on_history()

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


['Play it to the Bone (1999)',
 'Seven Beauties (Pasqualino Settebellezze) (1976)',
 "Swept Away (Travolti da un insolito destino nell'azzurro mare d'Agosto) (1975)",
 'Muse, The (1999)',
 'Carriers Are Waiting, The (Les Convoyeurs Attendent) (1999)']

In [34]:
# Recommendation based on this movie
recommender.recommend_on_movie('Money Train (1995)')

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


['Shadow, The (1994)',
 'Black Mask (Hak hap) (1996)',
 'Stranger, The (1994)',
 'Shanghai Noon (2000)',
 'Thunderball (1965)']

In [35]:
# Recommendation based on past watched movies, and this time four movies is there in the history.
recommender.recommend_on_history()

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


['Get Shorty (1995)',
 'Buffalo 66 (1998)',
 'Faster Pussycat! Kill! Kill! (1965)',
 'Lethal Weapon 4 (1998)',
 'Lethal Weapon 2 (1989)']

In [36]:
# Recommendation based on this movie
recommender.recommend_on_movie('GoldenEye (1995)')

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


['Anaconda (1997)',
 'Clear and Present Danger (1994)',
 'Surviving the Game (1994)',
 'Chain Reaction (1996)',
 'Rock, The (1996)']

In [37]:
# Recommendation based on past watched movies, and this time five movies is there in the history.
recommender.recommend_on_history()

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


['Runaway Train (1985)',
 'Daylight (1996)',
 'Con Air (1997)',
 'Fire Down Below (1997)',
 'Outbreak (1995)']

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
